In [1]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('tf_flowers', as_supervised=True, with_info=True)
dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes

2022-07-25 21:17:11.762295: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "INTERNAL: Couldn't parse JSON response from OAuth server.".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-25 21:17:24.802802: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-25 21:17:24.803405: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    'tf_flowers',
    split=['train[:10%]', 'train[10%:25%]', 'train[25%:]'],
    as_supervised=True
)

In [16]:
import tensorflow as tf
from tensorflow import keras

def preprocess(image,label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [17]:
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set_raw.map(preprocess).batch(batch_size).prefetch(1)
val_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [18]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)

avg = keras.layers.GlobalMaxPool2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [19]:
for layer in base_model.layers:
    layer.trainable=False

In [20]:
optimizer=keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.1)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_set, validation_data=val_set, epochs=5)

Epoch 1/5


2022-07-25 21:43:43.377610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 26.0131 - accuracy: 0.7427

2022-07-25 21:44:02.852408: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 25s 271ms/step - loss: 26.0131 - accuracy: 0.7427 - val_loss: 13.8486 - val_accuracy: 0.8240
Epoch 2/5
86/86 [==============================] - 22s 262ms/step - loss: 7.6132 - accuracy: 0.8812 - val_loss: 10.5259 - val_accuracy: 0.8494
Epoch 3/5
86/86 [==============================] - 23s 263ms/step - loss: 5.8708 - accuracy: 0.8928 - val_loss: 9.0885 - val_accuracy: 0.8584
Epoch 4/5
86/86 [==============================] - 22s 261ms/step - loss: 4.9850 - accuracy: 0.9012 - val_loss: 8.5037 - val_accuracy: 0.8494
Epoch 5/5
86/86 [==============================] - 22s 261ms/step - loss: 4.3596 - accuracy: 0.9106 - val_loss: 8.1105 - val_accuracy: 0.8530


In [12]:
for layer in base_model.top:
    layer.trainable=True

In [21]:
optimizer = keras.optimizers.SGD(lr = 0.001, momentum=0.9, decay = 0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_set, validation_data=val_set, epochs=5)

Epoch 1/5


2022-07-25 21:46:01.516645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 3.9417 - accuracy: 0.9132

2022-07-25 21:46:20.865852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 25s 270ms/step - loss: 3.9417 - accuracy: 0.9132 - val_loss: 8.0724 - val_accuracy: 0.8603
Epoch 2/5
86/86 [==============================] - 23s 262ms/step - loss: 3.8672 - accuracy: 0.9146 - val_loss: 8.0223 - val_accuracy: 0.8621
Epoch 3/5
86/86 [==============================] - 22s 261ms/step - loss: 3.8002 - accuracy: 0.9146 - val_loss: 7.9734 - val_accuracy: 0.8621
Epoch 4/5
86/86 [==============================] - 23s 263ms/step - loss: 3.7387 - accuracy: 0.9150 - val_loss: 7.9275 - val_accuracy: 0.8621
Epoch 5/5
86/86 [==============================] - 23s 265ms/step - loss: 3.6819 - accuracy: 0.9146 - val_loss: 7.8844 - val_accuracy: 0.8621


In [25]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('bset_xception.h5', save_best_only=True)
history = model.fit(train_set, validation_data=val_set, epochs=1, callbacks=[checkpoint_cb])

86/86 [==============================] - 23s 264ms/step - loss: 3.6293 - accuracy: 0.9161 - val_loss: 7.8440 - val_accuracy: 0.8603


In [26]:
model.evaluate(test_set)

12/12 [==============================] - 3s 221ms/step - loss: 9.0498 - accuracy: 0.8556


[9.049829483032227, 0.8555858135223389]

In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                         32)                                                         

In [28]:
for layer in base_model.layers:
    layer.trainable=True

In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                         32)                                                         

In [30]:
optimizer = keras.optimizers.SGD(lr = 0.001, momentum=0.9, decay = 0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_set, validation_data=val_set, epochs=5)

Epoch 1/5


/Applications/miniconda3/envs/geron/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-07-25 21:53:08.161897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 1.8251 - accuracy: 0.5378

2022-07-25 21:54:17.112420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 76s 840ms/step - loss: 1.8251 - accuracy: 0.5378 - val_loss: 0.7345 - val_accuracy: 0.7477
Epoch 2/5
86/86 [==============================] - 72s 832ms/step - loss: 0.4569 - accuracy: 0.8721 - val_loss: 0.4713 - val_accuracy: 0.8457
Epoch 3/5
86/86 [==============================] - 71s 832ms/step - loss: 0.1650 - accuracy: 0.9531 - val_loss: 0.4512 - val_accuracy: 0.8621
Epoch 4/5
86/86 [==============================] - 71s 830ms/step - loss: 0.0434 - accuracy: 0.9855 - val_loss: 0.5325 - val_accuracy: 0.8603
Epoch 5/5
86/86 [==============================] - 72s 835ms/step - loss: 0.0184 - accuracy: 0.9964 - val_loss: 0.6139 - val_accuracy: 0.8639


In [31]:
model.evaluate(test_set)

12/12 [==============================] - 3s 234ms/step - loss: 0.5058 - accuracy: 0.8801


[0.5058313608169556, 0.8801089525222778]